**Use of Grid Search with Random Forest on the Boston Housing data**

In [1]:
# Import the usual suspects
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# fit2ec2
import fit2ec2 as fe

<IPython.core.display.Javascript object>

**Load and Split data**

In [3]:
data = pd.read_csv('data/data.csv')

y = np.log(data.SalePrice)
X = data.drop(['SalePrice', 'Id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
                          X, y, random_state=42, test_size=.33)

print(X_train.shape)

(978, 246)


**Initialize the Random Forest and the Grid Search**

In [4]:
# Initialize Random Forest
rf = RandomForestRegressor()

In [5]:
# Choose parameters for Grid Search
rf_params = {'n_estimators': range(150, 300, 10), 'max_depth': range(3, 15, 1),'min_samples_leaf':[0.04,0.06,0.08], 'max_features':[0.2,0.4,0.6,0.8]}

In [6]:
grid = GridSearchCV(estimator = rf,
                         param_grid = rf_params,
                         cv=2,
                         scoring='neg_mean_squared_error',
                         verbose=1)

**Initialize Fit2ec2**

In [7]:
cmp = fe.Compute()

# Or specify the private key name
# cmpC = fe.Fit2ec2(keyname="mykey")

**Create the ec2 instance**

In [8]:
# Create an EC2 with the default AWS Linux AMI and a type of t2.micro
cmp.create()

# Or specify the image id (AMI ID) and the instance type (see https://aws.amazon.com/ec2/instance-types/)
#cmp.create(imageId='ami-06ce3edf0cff21f07', instanceType='t2.micro')

Creating key pair [ec2-keypair-20200509-173838.pem]...
Key pair created!
Creating EC2 instance with type [t2.micro]
Wait for EC2 instance to  be created...
EC2 instance created!
	Id: i-0d76aa5e7bb20dc73
	Public IP address: 34.242.70.238
Generating project Python requirements


**Fit the model with the data on the ec2 instance**

In [9]:
# Better to wait 1 min before running that command
grid = cmp.fit(grid, X_train, y_train)

Saving model.pickle
Saving X.pickle
Saving y.pickle
Transfering [./temp/X.pickle] to ec2
Transfering [./temp/y.pickle] to ec2
Transfering [run.sh] to ec2
Transfering [./temp/model.py] to ec2
Transfering [./temp/requirements.txt] to ec2
Transfering [./temp/model.pickle] to ec2
Executing [chmod 700 run.sh]
Out: 
Error: 
Executing [./run.sh]

Stopping remote processes. (run terminate to destroy the ec2 instance)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Executing [pgrep run.sh | xargs pkill -9 -P]
Out: 
Error: 
Traceback (most recent call last):
  File "/Users/odyssee/git/fit2ec2/fit2ec2.py", line 157, in fit
    self.__remote_exec(["chmod 700 run.sh","./run.sh"])
  File "/Users/odyssee/git/fit2ec2/fit2ec2.py", line 110, in __remote_exec
    print("Out: {}".format(stdout.read().decode('unicode_escape')))
  File "/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/paramiko/file.py", line 200, in read
    new_data = self._read(self._DEFAULT_BUFSIZE)
  File "/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/paramiko/channel.py", line 1361, in _read
    return self.channel.recv(size)
  File "/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/paramiko/channel.py", line 699, in recv
    out = self.in_buffer.read(nbytes, self.timeout)
  File "/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/paramiko/buffered_pipe.py", line 160, in read
    self._cv.wait

SystemExit: 0

/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Terminate the ec2 instance and remove the private key**

In [10]:
cmp.terminate()

Deleting remote key pair...
Terminate EC2 instance [i-0d76aa5e7bb20dc73]...
Delete local key pair file...
Done!
